# Zadania - metodologia testowania

### Walidacja krzyżowa

1. (15 pkt) Zaimplementuj 4-krotną walidację krzyżową na danych treningowych ze zbioru MNIST. W jej wyniku powinny powstać cztery różne modele $M_i$ dla $i=1\dots4$. Dla każdego modelu dobierz wspólne początkowe parametry ($\alpha$, rozmiar wsadu, liczba epok), zastosuj randomizację danych między epokami.
1. (10 pkt) Oblicz jakość każdego modelu na jego zbiorze walidacyjnym oraz średnią poprawność klasyfikacyjną na całym zbiorze treningowym poprzez uśrednienie wyników każdego z czterech zbiorów walidacyjnych.
1. (15 pkt) Dobierz najlepsze parametry dla modeli $M_i$ na podstawie skuteczności osiąganej na zbiorze treningowym. Podaj skuteczności, podobnie jak w punkcie 2. 
1. (10 pkt) Zbuduj model z tak dobranymi parametrami (uśrednij, jeżeli są różne dla $M_i$) na całych danych treningowych i sprawdź jego skuteczność na zbiorze testowym. Czy model z takimi parametrami da najlepszy możliwy wynik na zbiorze testowym? Odpowiedź uzasadnij.

In [36]:
import numpy as np
import itertools
import pprint as pp

def safeSigmoid(x, eps=0):
    y = 1.0/(1.0 + np.exp(-x))
    if eps > 0:
        y[y < eps] = eps
        y[y > 1 - eps] = 1 - eps
    return y

def h(theta, X, eps=0.0):
    return safeSigmoid(X*theta, eps)

def J(h,theta,X,y):
    m = len(y)
    f = h(theta, X, eps=10**-7)
    return -np.sum(np.multiply(y, np.log(f)) + 
                   np.multiply(1 - y, np.log(1 - f)), axis=0)/m

def dJ(h,theta,X,y):
    return 1.0/len(y)*(X.T*(h(theta,X)-y))

def softmax(X):
    return np.exp(X)/np.sum(np.exp(X))

In [37]:
def SGD(h, fJ, fdJ, theta, X, y, 
        alpha=0.001, maxEpochs=1, batchSize=100, shuffle=False):
    m, n = X.shape
    start, end = 0, batchSize
    
    maxSteps = (m * float(maxEpochs)) / batchSize
    for i in range(int(maxSteps)):
        if shuffle and (i * batchSize) % m == 0:
            indexes = [j for j in range(len(X))]
            np.random.shuffle(indexes)
            X = X[indexes]
            y = y[indexes]
            
        XBatch, yBatch =  X[start:end,:], y[start:end,:]

        theta = theta - alpha * fdJ(h, theta, XBatch, yBatch)
        
        if start + batchSize < m:
            start += batchSize
        else:
            start = 0
        end = min(start + batchSize, m)
        
    return theta

In [38]:
import os
import struct
import numpy as np
import itertools
from statistics import mode

%matplotlib inline

def readMnist(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

def showImage(image):
    """
    Render a given numpy.uint8 2D array of pixel data.
    """
    from matplotlib import pyplot
    import matplotlib as mpl
    fig = pyplot.figure()
    ax = fig.add_subplot(1,1,1)
    imgplot = ax.imshow(image, cmap=mpl.cm.Greys)
    imgplot.set_interpolation('nearest')
    ax.xaxis.set_ticks_position('top')
    ax.yaxis.set_ticks_position('left')
    pyplot.show()
    
def mnistMatrix(data, maxItems=1000):
    datalist = [t for t in data]
    m = maxItems if maxItems != -1 else len(datalist)
    n = 28 * 28 + 1
    X = np.matrix(np.zeros(m * n)).reshape(m, n)
    Y = np.matrix(np.zeros(m)).reshape(m, 1)
    for i, (label, image) in enumerate(datalist[:m]):
        X[i, 0] = 1 # bias term
        X[i, 1:] = image.reshape(28*28,)
        Y[i] = label
    return X, Y

def normalize(X):
    X[:,1:] = X[:,1:] / 255
    return X

def mapY(y, cls):
    m = len(y)
    yBi = np.matrix(np.zeros(m)).reshape(m, 1)
    yBi[y == cls] = 1.
    return yBi

def indicatorMatrix(y):
    classes = np.unique(y.tolist())
    m = len(y)
    k = len(classes)
    Y = np.matrix(np.zeros((m, k)))
    for i, cls in enumerate(classes):
        Y[:,i] = mapY(y, cls)
    return Y

def classify(thetas, X, debug=False):
    regs = np.array([(X*theta).item() for theta in thetas])
    if debug:
        print("regs  =", regs)
    probs = softmax(regs)
    if debug:
        print("probs =", np.around(probs, decimals=3))
    return np.argmax(probs), probs

def trainMaxEnt(X, Y, alpha = 1.0, maxEpochs = 2, batchSize = 50, shuffle = False):
    n = X.shape[1]
    thetas = []
    for c in range(Y.shape[1]):
        YBi = Y[:,c]
        theta = np.matrix(np.random.random(n)).reshape(n,1)
        thetaBest = SGD(h, J, dJ, theta, X, YBi, alpha, maxEpochs, batchSize, shuffle)
        thetas.append(thetaBest)
    return thetas

def calculateAcc(thetas, X_test, y_test, debug=True):
    acc = 0.0
    for i in range(len(y_test)):
        cls, probs = classify(thetas, X_test[i])
        correctCls = int(y_test[i].item())
        
        acc += correctCls == cls
    return acc

## Walidacja krzyżowa

In [39]:
def divide_into_parts(X, y, amount):
    chunk = int(round(len(X)/amount))
    X_parts = [X[i:i+chunk] for i in range(0, len(X), chunk)]
    y_parts = [y[i:i+chunk] for i in range(0, len(y), chunk)]
    
    return X_parts, y_parts

X, y = mnistMatrix(readMnist(dataset='training'), maxItems=-1)
X_test, y_test = mnistMatrix(readMnist(dataset='testing'), maxItems=-1)
X_norm = normalize(X)
X_test_norm = normalize(X_test)

In [40]:
indexes = [j for j in range(len(X))]
np.random.shuffle(indexes)
X_norm = X_norm[indexes]
y = y[indexes]

X_parts, y_parts = divide_into_parts(X_norm, y, 4)
thetas = []
results = []
for i in range(len(X_parts)):
    D_x = X_parts[i]
    D_y = y_parts[i]
    S_x = np.concatenate([X_parts[j] for j in range(len(X_parts)) if j != i])
    S_y = indicatorMatrix(np.concatenate([y_parts[j] for j in range(len(y_parts)) if j != i]))
    theta = trainMaxEnt(S_x, S_y, alpha = 0.1, maxEpochs = 1, batchSize = 100, shuffle = True)
    acc = calculateAcc(theta, D_x, D_y)/len(D_x)
    print("Accuracy model %d = %f" % (i, acc))
    thetas.append(theta)
    results.append(acc)

Accuracy model 0 = 0.849200
Accuracy model 1 = 0.854067
Accuracy model 2 = 0.853400
Accuracy model 3 = 0.852267


In [41]:
acc_mean = np.mean(results)
print("Accuracy total model = %f" % acc)

Accuracy total model = 0.852267


In [42]:
alphas = [0.1, 0.5, 1.0, 2.0]
epochs = [2, 3, 1]
batches = [50, 100, 10, 20]

parameters = list(itertools.product(alphas, epochs, batches))
best_parameters = []

for i in range(len(X_parts)):
    D_x = X_parts[i]
    D_y = y_parts[i]
    S_x = np.concatenate([X_parts[j] for j in range(len(X_parts)) if j != i])
    S_y = indicatorMatrix(np.concatenate([y_parts[j] for j in range(len(y_parts)) if j != i]))
    results = []

    for parameter in parameters:
        alpha, epoch, batch_size = parameter
        theta = trainMaxEnt(S_x, S_y, alpha = alpha, maxEpochs = epoch, batchSize = batch_size, shuffle = True)
        results.append((parameter, calculateAcc(theta, D_x, D_y)/len(D_x)))
    
    best_result = max(results, key = lambda item:item[1])
    best_parameters.append(best_result[0])
    print("Best accuracy for model %d and parameters %s = %f" % (i, str(best_result[0]), best_result[1]))

Best accuracy for model 0 and parameters (1.0, 3, 100) = 0.908400
Best accuracy for model 1 and parameters (0.5, 2, 20) = 0.906667
Best accuracy for model 2 and parameters (0.5, 2, 20) = 0.905000
Best accuracy for model 3 and parameters (0.1, 3, 20) = 0.902533


In [43]:
mean_parameters = np.mean(best_parameters, axis=0)
alpha, epoch, batch = mean_parameters
epoch, batch = int(epoch), int(batch)
y_ind = indicatorMatrix(y)

theta = trainMaxEnt(X_norm, y_ind, alpha = alpha, maxEpochs = epoch, batchSize = batch, shuffle = True)
print("Accuracy total model for parameters %s = %f" % ((alpha, epoch, batch), calculateAcc(theta, X_test_norm, y_test)/len(X_test_norm)))

Accuracy total model for parameters (0.52500000000000002, 2, 40) = 0.911800


Nie wydaje mi się, aby tak zbudowany model był najlepszy możliwy. A to dlatego, że nie zawsze uśrednianie parametrów będzie dawało najlepszą skuteczość. Łatwo wyobrazić sobie sytuację, gdy dla prawie wszystkich modeli najlepsze będą te same parametry, a dla jednego inne. Wówczas lepszą skuteczność uzyskalibyśmy biorąc te parametry, które w większości przypadków okazały się prowadzić do dobrego rozwiązania. Również zastosowanie średniej ważonej (wagi w zależności od liczby modeli, dla których dany zestaw parametrów okazał się najlepszy) byłoby lepszym pomysłem niż średnia arytmetyczna.